###  1. Importaciones y corpus

Importamos PyTorch y utilidades, y definimos un corpus mínimo para demostrar Word2Vec (Skip‑Gram).


####  ¿Qué intenta aprender Word2Vec?

Word2Vec entrena un modelo predictivo muy simple: dado un término objetivo `w_t`, intenta asignarle una representación vectorial \(\vec{v}_{w_t}\) capaz de predecir qué palabras pueden aparecer a su alrededor. En la variante Skip-Gram que usamos aquí, maximizamos la probabilidad condicional `P(contexto | palabra_objetivo)` sobre todo el corpus. Al proyectar cada palabra a un espacio continuo de baja dimensión, términos que comparten contexto terminan con vectores cercanos en ese espacio, lo cual captura similitudes semánticas y sintácticas.

En la práctica, la red neuronal consta de dos capas lineales: la primera actúa como tabla de embeddings y la segunda proyecta de vuelta al vocabulario. Al entrenar con `CrossEntropyLoss`, la capa de salida aprende una distribución tipo softmax sobre todas las palabras del vocabulario y propaga gradientes hacia la tabla de embeddings, afinando los vectores para cumplir el objetivo predictivo. Este montaje hace que Word2Vec sea ligero pero muy efectivo para aprender representaciones distribuidas.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random

# torch administra tensores y operaciones en CPU o GPU
# nn proporciona bloques reutilizables para construir arquitecturas
# optim reúne los optimizadores clásicos (usaremos Adam)
# Dataset y DataLoader ordenan el acceso por lotes a los pares (target, contexto)
# random es útil para fijar semillas cuando necesitemos reproducibilidad   

In [2]:
# Corpus diminuto para mantener la demostración completamente interpretable
corpus = [
    "el gato come pescado",
    "el perro come carne",
    "el pájaro vuela alto",
    "el pez nada en el agua"
]

# En un proyecto real se sustituiría por millones de oraciones para captar mejor la estadística del lenguaje

###  2. Preprocesamiento

- Tokenizamos el corpus (separamos palabras)
- Construimos el vocabulario
- Creamos los mapas palabra↔índice (one‑hot implícito)


####  ¿Qué logramos con este preprocesamiento?

1. **Tokenización:** dividimos cada oración en palabras para conservar el orden y poder deslizar una ventana de contexto.
2. **Vocabulario ordenado:** al convertir el conjunto de tokens en una lista ordenada garantizamos índices deterministas para cada palabra.
3. **Mapas bidireccionales:** `word_to_idx` y `idx_to_word` nos permiten ir y volver entre representaciones simbólicas y numéricas, requisito para alimentar la red y para interpretar los embeddings al final.
4. **Tamaño del vocabulario:** `vocab_size` define cuántas filas tendrá la tabla de embeddings y cuántos logits producirá la capa de salida.

Con esta base, cada palabra queda representada como un índice entero que hará las veces de one-hot implícito cuando consultemos la capa `nn.Embedding`.


In [3]:
# Tokenizamos el corpus
tokens = [sentence.split() for sentence in corpus]
vocab = sorted(set(sum(tokens, [])))

# Mapas palabra <-> índice
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for word, i in word_to_idx.items()}

vocab_size = len(vocab)
print("Vocabulario:", vocab)

Vocabulario: ['agua', 'alto', 'carne', 'come', 'el', 'en', 'gato', 'nada', 'perro', 'pescado', 'pez', 'pájaro', 'vuela']


###  3. Crear dataset con ventanas de contexto (Skip‑Gram)

Ventana de tamaño 2: para cada palabra objetivo, tomamos hasta 2 palabras antes y 2 después como contexto. Generamos pares (target, context).


####  Razonamiento del muestreo Skip-Gram

- Recorremos cada oración con una ventana simétrica de tamaño `window_size`.
- Tratamos cada palabra central como la señal y las palabras vecinas como etiquetas.
- Al repetir este proceso en todo el corpus generamos un conjunto explícito de ejemplos `(palabra_objetivo, palabra_contexto)` listos para aprendizaje supervisado.
- La densidad de ejemplos depende del tamaño de la ventana: ventanas grandes capturan relaciones temáticas, ventanas pequeñas capturan relaciones sintácticas.

Este muestreo es equivalente a maximizar la suma de log-probabilidades `\sum_t \sum_{c \in C_t} \log P(c | w_t)`, donde `C_t` es el conjunto de contextos válidos alrededor de `w_t`. Así entrenamos a la red para que cada vector contenga información predictiva sobre su vecindario lingüístico.


In [4]:
def generate_skipgram_pairs(tokens, window_size=2):
    """Genera pares (palabra_objetivo, palabra_contexto) recorriendo ventanas Skip-Gram."""
    pairs = []
    for sentence in tokens:
        for i, target in enumerate(sentence):
            for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
                if i != j:
                    # Excluimos la palabra central y añadimos únicamente los vecinos válidos
                    pairs.append((target, sentence[j]))
    return pairs

pairs = generate_skipgram_pairs(tokens, window_size=2)
print("Ejemplo de pares (target, context):", pairs[:10])
# Observa cómo cada palabra aparece tantas veces como ventanas la incluyan

Ejemplo de pares (target, context): [('el', 'gato'), ('el', 'come'), ('gato', 'el'), ('gato', 'come'), ('gato', 'pescado'), ('come', 'el'), ('come', 'gato'), ('come', 'pescado'), ('pescado', 'gato'), ('pescado', 'come')]


###  4. Dataset en PyTorch

Empaquetamos los pares (target, context) en un `Dataset` y un `DataLoader` para entrenamiento por lotes.


In [5]:
def generate_skipgram_pairs(tokens, window_size=2):
    pairs = []
    for sentence in tokens:
        for i, target in enumerate(sentence):
            for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
                if i != j:
                    pairs.append((target, sentence[j]))
    return pairs

pairs = generate_skipgram_pairs(tokens, window_size=2)
print("Ejemplo de pares (target, context):", pairs[:10])

Ejemplo de pares (target, context): [('el', 'gato'), ('el', 'come'), ('gato', 'el'), ('gato', 'come'), ('gato', 'pescado'), ('come', 'el'), ('come', 'gato'), ('come', 'pescado'), ('pescado', 'gato'), ('pescado', 'come')]


###  5. Modelo Word2Vec (Skip‑Gram)

Arquitectura mínima:
- Capa de embeddings (convierte índices→vectores)
- Capa lineal de salida (vocabulario) con softmax implícito en la pérdida.


####  Desglose de la arquitectura

- `nn.Embedding(vocab_size, embedding_dim)` es conceptualmente una matriz `W_in` de tamaño `(vocab_size, embedding_dim)`; cada fila es el vector asociado a una palabra.
- La capa lineal `nn.Linear(embedding_dim, vocab_size)` actúa como `W_out`, que proyecta el embedding al espacio del vocabulario para producir un logit por palabra.
- Durante el *forward*, seleccionamos la fila correspondiente a cada palabra del lote, la multiplicamos por `W_out` y obtenemos una distribución sin normalizar.
- La pérdida `CrossEntropyLoss` aplica softmax + log-loss, forzando que el logit de la palabra de contexto correcta sea mayor que los demás.

Así, los embeddings se mueven en la dirección que maximiza la probabilidad de observar vecinos reales y minimiza la de vecinos incorrectos.


In [6]:
class Word2VecDataset(Dataset):
    """Envuelve los pares Skip-Gram y los expone como tensores listos para PyTorch."""

    def __init__(self, pairs, word_to_idx):
        self.pairs = pairs
        self.word_to_idx = word_to_idx

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        target, context = self.pairs[idx]
        target_idx = torch.tensor(self.word_to_idx[target], dtype=torch.long)
        context_idx = torch.tensor(self.word_to_idx[context], dtype=torch.long)
        return target_idx, context_idx

dataset = Word2VecDataset(pairs, word_to_idx)
# DataLoader baraja los pares en cada epoch y agrupa 8 ejemplos por lote
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

### ⚙️ 6. Entrenamiento

Optimizamos con `CrossEntropyLoss` y `Adam`. El objetivo es predecir la palabra de contexto dada la palabra objetivo.

**Ciclo de entrenamiento simplificado:**
1. Obtenemos embeddings para cada palabra del lote.
2. Calculamos logits para todas las palabras del vocabulario y derivamos una distribución softmax implícita.
3. Evaluamos la entropía cruzada entre dicha distribución y el índice real del contexto.
4. Retropropagamos los gradientes para ajustar tanto la tabla de embeddings como la capa de salida.

Aunque este ejemplo usa *full softmax*, en corpora grandes se acostumbra a combinarlo con *negative sampling* o *hierarchical softmax* para reducir el costo computacional.


In [7]:
class Word2VecModel(nn.Module):
    """Versión mínima del modelo Skip-Gram con embeddings aprendibles."""

    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.output = nn.Linear(embedding_dim, vocab_size)
    
    def forward(self, target_word):
        emb = self.embeddings(target_word)  # Recuperamos los vectores de cada palabra del lote
        logits = self.output(emb)  # Proyectamos a vocab_size logits para evaluar cada posible contexto
        return logits

###  7. Ver los embeddings finales

Inspeccionamos la matriz de embeddings (una fila por palabra). Palabras con contextos similares tendrán vectores cercanos.


In [8]:
embedding_dim = 8  # Longitud de cada vector semántico
model = Word2VecModel(vocab_size, embedding_dim)

criterion = nn.CrossEntropyLoss()  # Softmax + log-loss en un paso estable
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 200
model.train()

for epoch in range(epochs):
    total_loss = 0.0
    for target, context in dataloader:
        optimizer.zero_grad()  # Limpiamos gradientes acumulados
        logits = model(target)
        loss = criterion(logits, context)
        loss.backward()  # Retropropagamos el error hacia las dos matrices de pesos
        optimizer.step()  # Actualizamos W_in y W_out simultáneamente
        total_loss += loss.item()
    if (epoch + 1) % 40 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

Epoch 40/200, Loss: 9.1628
Epoch 80/200, Loss: 8.4335
Epoch 120/200, Loss: 8.3221
Epoch 160/200, Loss: 8.2862
Epoch 200/200, Loss: 8.2487


###  En General

- La ventana de contexto se desplaza palabra por palabra.
- Generamos pares (target, contexto) para aprendizaje supervisado.
- El modelo aprende a predecir el contexto a partir de la palabra central.
- Los pesos de la capa de embeddings son las representaciones vectoriales (significado) de las palabras.
- Podemos medir similitud semántica con métricas como coseno o distancia euclidiana sobre esos vectores.
- Al promediar embeddings de frases podemos construir características para modelos posteriores.

Una vez entrenado, basta con congelar `model.embeddings` y reutilizarlo como extractor de características en tareas de clasificación, clustering o visualización (p.ej. reduciendo de 8D a 2D con UMAP o t-SNE).


####  ¿Cómo interpretar la matriz de embeddings?

- Cada fila corresponde a una palabra en el orden definido por `word_to_idx`.
- Componentes positivos/negativos indican afinidades latentes aprendidas durante el entrenamiento.
- Para comparar palabras, calcula el coseno entre dos filas (`torch.nn.functional.cosine_similarity`).
- También es posible proyectar la matriz a 2D y visualizar clusters para validar si animales, acciones, etc. quedan agrupados.


In [9]:
embeddings = model.embeddings.weight.data  # Tensores con las coordenadas finales
for word, idx in word_to_idx.items():
    print(f"{word:10s} -> {embeddings[idx].tolist()}")

# Puedes almacenar esta matriz para cálculos posteriores o para visualizarla con herramientas como TensorBoard Projector

agua       -> [-0.2564436197280884, 1.5151938199996948, -0.3949437141418457, -2.2353787422180176, -1.5946879386901855, -1.170777678489685, -1.455666184425354, 0.8406234383583069]
alto       -> [-0.23509903252124786, -1.768825650215149, 0.650020956993103, 1.8077776432037354, 0.2962290346622467, -1.962864637374878, 0.3335600197315216, 2.5872435569763184]
carne      -> [2.601318597793579, 0.222939595580101, 1.8648544549942017, 0.34594205021858215, 1.909814476966858, 0.4502359926700592, 0.20215937495231628, -1.1665366888046265]
come       -> [0.5047829151153564, 2.1917102336883545, 0.21772634983062744, -2.2317585945129395, 0.5539979934692383, 1.4984265565872192, -1.1155365705490112, -0.2711646854877472]
el         -> [-1.2901389598846436, -0.9246841073036194, 2.1643381118774414, -0.2278233915567398, 0.07226025313138962, -0.40380069613456726, 1.1552389860153198, -0.1157672330737114]
en         -> [-0.15550340712070465, -1.967090129852295, -3.3128697872161865, 0.16994978487491608, -1.3420979